# 1. Organizing

## 1.1. Subqueries

In [ ]:
select employee.first_name, employee.last_name
from employee
where employee.emp_id in (
    select works_with.emp_id
    from works_with
    where works_with.total_sales > 30000
);

## 1.2. Common tabel expression (CTE)

In [ ]:
WITH first_term as
(
    SELECT id_bioguide
    ,min(term_start) as first_term
    FROM legislators_terms
    GROUP BY 1
)
SELECT date_part('year',age(b.term_start,a.first_term)) as periods
,count(distinct a.id_bioguide) as cohort_retained
FROM first_term a
JOIN legislators_terms b on a.id_bioguide = b.id_bioguide
GROUP BY 1
;
/*If a CTE is involved, however, all of the surrounding
code must be commented out first.*/

In [ ]:
-- We have multiple queries with one 'WITH'
WITH aa as (
),
bb as ()

## 1.3. Statements and order of execution

1. FROM, JOIN
2. WHERE
3. GROUP BY
4. aggregate functions
5. HAVING
6. window functions
7. SELECT
8. DISTINCT
9. UNION/INTERSECT/EXCEPT
10. ORDER BY
11. OFFSET
12. LIMIT/FETCH/TOP

![alt text](image/order.png)

**Note that:**
- Aggregate functions can be within HAVING, but not in WHERE
- You can use window functions in SELECT and ORDER BY. However, you can’t put window functions anywhere in the FROM, WHERE, GROUP BY, or HAVING clauses
- ORDER BY can not be writen within views, CTEs, or subqueries
- In GROUP BY, ORDER BY clause, we can use number to call columns (in SELECT)

### JOIN

#### Non-equality JOIN

In [ ]:
SELECT
a.toy_name AS toy_a,
b.toy_name AS toy_b
FROM toy a
JOIN toy b
ON a.cat_id < b.cat_id;

#### Multiple conditions JOIN

In [ ]:
SELECT
	cat_name,
	o.name AS owner_name,
	c.age AS cat_age,
	o.age AS owner_age
FROM cat c
JOIN owner o ON c.owner_id = o.id
	AND c.age < o.age;

select s.name as name
        ,s.marks as mark
        ,g.grade as grade
    from students s
    left join grades g on s.marks between g.min_mark and g.max_mark
    -- s.marks > g.min_mark and s.marks < g.max_mark

#### List of all combination with JOIN

In [ ]:
select distinct a.emp_name
	,b.month_no
from od_table a
join (select distinct extract(month from order_dt) as month_no from od_table) b
	on 1=1
where emp_name is not null
order by 1,2

#### Multiple JOIN

First, two tables are joined, then the third table is joined to the result of the previous joining

### GROUP BY

The `GROUP BY` clause is required when the SELECT clause contains aggregations and at least one nonaggregated field. An easy way to remember what should go in the GROUP BY clause is that it should have **every field that is not part of an aggregation**

### UNION

#### UNION vs UNION ALL

- UNION removes duplicates
- UNION ALL is faster, since the database doesn’t have to do a pass over the data to find duplicates

#### Unpivoting with UNION

**Use `UNION` to combine datasets from multiple queries**

- The numbers of columns and datatypes of each component queries must match
- The columns name come from the the first query

Eg:
![alt text](image/image.png)

In [ ]:
SELECT country, '1980' as year, year_1980 as population FROM country_populations
UNION ALL
SELECT country, '1990' as year, year_1990 as population FROM country_populations
UNION ALL
SELECT country, '2000' as year, year_2000 as population FROM country_populations
UNION ALL
SELECT country, '2010' as year, year_2010 as population FROM country_populations;

# 2. Functions

## 2.1. Numeric functions

### Exponential

In [ ]:
square() -- square
POWER(base, exponent) -- exponentiation
SQRT(9) -- square root

## 2.2 CASE statement

In [ ]:
case    
  when condition1 then return_value_1
  when condition2 then return_value_2
  ...
  else return_value_default end

-- can write like this
case variable
	when value1 then return1
	when value2 then return2
	...
	else default end

### Pivot with CASE

In [ ]:
SELECT order_date
	,sum(case when product = 'shirt' then order_amount else 0 end) as shirts_amount
	,sum(case when product = 'shoes' then order_amount else 0 end) as shoes_amount
	,sum(case when product = 'hat' then order_amount else 0 end) as hats_amount
FROM orders GROUP BY 1;

SELECT gender, first_state
  ,max(case when period = 0 then pct_retained end) as yr0
  ,max(case when period = 2 then pct_retained end) as yr2
  ,max(case when period = 4 then pct_retained end) as yr4
  ,max(case when period = 6 then pct_retained end) as yr6
  ,max(case when period = 8 then pct_retained end) as yr8
  ,max(case when period = 10 then pct_retained end) as yr10
FROM sparse_cohort
WHERE first_state = 'AL'
GROUP BY 1,2;

## 2.3. Window functions

In [ ]:
window_function(column) OVER 
    (PARTITION BY partition_column -- separate each categories in the field
	ORDER BY order_column -- clause determines the ordering of the rows
	ROWS/RANGE frame_definition) 

-- aggregate
avg()
max() / min()
sum()
count()

-- ranking
row_number()
rank()
dense_rank()
percent()
percentile_cont()
ntile() / qcut()
cume_dist()

-- value
lag() / lead()
first_value() / last_value()
nth_value()

### `ntile`

In [ ]:
select customer_id, order_id, order_amount,
ntile(10) over (order by order_amount) as n_tile
from orders

### `percent_rank`

In [ ]:
percent_rank() over (order by order_amount) *100 as n_tile

### `lag/lead`

Carrying over a value from the previous row is called **fill forward**, while using a value from the next row is called **fill backward**. These can be accomplished with the `lag` and `lead` window functions, respectively

In [ ]:
LAG(expression [, offset])

-- LAG retrieves a value from a specified number of rows before the current row.
-- By default, it retrieves the value from the immediately preceding row (offset of 1).

### `percentile_cont`

In [ ]:
SELECT
  percentile_cont(0.25) within group (order by mag) as pct_25
  ,percentile_cont(0.5) within group (order by mag) as pct_50
  ,percentile_cont(0.75) within group (order by mag) as pct_75

# 3. Other syntax

## Operators

### Comparison operators

In [ ]:
=
<>
> >=
< <=

### Logical operators

In [ ]:
AND
OR 
IS
NOT

BETWEEN
-- Inclusive for both the lower and upper bounds
-- Eg: between 600 and 900

IN
-- Eg: 
    where name in ('Claire', 'Kate', 'Mike')

We can compare multiple variable: `(a, b, c) = (x, y, z)` or `(a, b, c) in (x, y, z)`

In [ ]:
select 
	w.id
    ,wp.age
    ,w.coins_needed
    ,w.power
from Wands w
join Wands_Property wp on w.code = wp.code 
where wp.is_evil = 0 
  and (wp.age, w.coins_needed, w.power) in (
    select wp.age, min(w.coins_needed), w.power
    from Wands w
    join Wands_Property wp
      on w.code = wp.code
    group by age , power)
order by 4 desc, 2 desc;

## Wildcards

In [ ]:
-- find any client's who are an LLC
select *
from client
where client_name LIKE '%LLC'; -- % = any number of characters, _ = 1 character

select *
from branch_supplier
where supplier_name LIKE '%Label%';

-- find any employee born in Oct
select * from employee
where birth_day like '____-10%';

-- find any clients who are schools
select *
from client
where client_name like '%School%';

-- tell SQL to treat wildcard characters as literal characters
select 'abc a%b abc' like '%\%%';
select 'abc a__b abc' like '%\_\_%';

# 4. Use case

## 4.1. Cleaning

### 4.1.1. Detecting duplicates

Use subquery and `where` clause

In [ ]:
SELECT *
FROM (
    SELECT customer_id, order_amount, sex, cashier, count(*) as records
    FROM orders
    GROUP BY 1,2,3,4
) a
WHERE records > 1;

Use `having` clause

In [ ]:
SELECT 
column_a, column_b, column_c..., count(*) as records
FROM...
GROUP BY 1,2,3...
HAVING count(*) > 1;

### 4.1.2. Handling duplicates

Use `distinct` or `group by` (usually with an aggregation)

### 4.1.3. Handling nulls

With `case`

In [ ]:
case when num_orders is null then 0 
	else num_orders end
	
case when address is null then 'Unknown' 
	else address end
	
case when column_a is null then column_b 
	else column_a end

With `coalesce` function

### 4.1.4. Dealing with missing data

**Fill missing data with a constant value**

For example, imagine there was a software bug that prevented filling the price for an item called “xyz,” but we know the price is always $20. A CASE statement can be added to the query to handle this:

In [ ]:
case when price is null and item_name = 'xyz' then 20
	else price
	end as price

**Fill with a derived value**

Either a mathematical function on other columns or a CASE statement

For example, we can calculate net_sales by subtracting discount from gross_sales:

In [ ]:
SELECT gross_sales - discount as net_sales...

**Fill with values from other rows**

**Lack of granularity**

For data that is available but not at the granularity needed (lack of granularity), we often have to ***create additional rows*** in the data set

Annual subscription example: We can spread this annual subscription amount into 12 equal monthly revenue amounts by dividing by 12:

In [ ]:
SELECT customer_id, subscription_date, annual_amount
, annual_amount / 12 as month_1, annual_amount / 12 as month_2
...,annual_amount / 12 as month_12
FROM customer_subscriptions;

We can use the date dimension to create a record for each month by `JOINing` the date dimension on dates that are between the subscription_date and 11 months later (for 12 total months):

In [ ]:
SELECT a.date
,b.customer_id,b.subscription_date,b.annual_amount / 12 as monthly_subscription
FROM date_dim a
JOIN customer_subscriptions b on a.date between b.subscription_date
and b.subscription_date + interval 11 month;

## 4.2. Data transformations

With `case`

`CASE` can clean, enrich, flag or add dummy variables

In [ ]:
CASE when gender = 'F' then 'Female'
	when gender = 'female' then 'Female'
	when gender = 'femme' then 'Female'
	else gender
	end as gender_cleaned
	
-- CASE statements can consider multiple columns and can contain AND/OR logic. 
-- They can also be nested, though often this can be avoided with AND/OR logic:

case 
	when likelihood <= 6 and country = 'US' and high_value = true then 'US high value detractor'
	when likelihood >= 9 and (country in ('CA','JP') or high_value = true) then 'some other label'
	... end

### Binning

**Binning intro**

Binning → work with continuous values

Ranges of values are grouped together → ***bins or buckets***

Arbitrary-sized bins <> Fixed-sized bins (rounding, logarithms, n-tiles)

#### Binning with `case`

In [ ]:
SELECT
    case
        when order_amount <= 100 then 'up to 100'
        when order_amount <= 500 then '100 - 500'
        else '500+' end as amount_bin,
    case
        when order_amount <= 100 then 'small'
        when order_amount <= 500 then 'medium'
        else 'large' end as amount_category,
    count(customer_id) as customers
FROM orders GROUP BY 1,2;

#### Binning with `round()` (rounding)

In [ ]:
SELECT round(sales,-1) as bin,
    count(customer_id) as customers
    FROM table GROUP BY 1;

#### Binning with logarithms

In [ ]:
SELECT log(sales) as bin,
    count(customer_id) as customers
    FROM table GROUP BY 1;

### Type conversions and casting

Converting an integer to a string can be useful in `CASE` statements when categorizing numeric values with some unbounded upper or lower value

In [ ]:
case when order_items <= 3 then order_items::varchar
	else '4+'
	end

Type conversions is also useful when **values should be integers are parsed out of a string**, then we want to aggregate or use mathematical functions on them

In [ ]:
replace('$19.99','$','')::float

cast(replace('$19.99','$','')) as float

Imagine that transaction/event data often arrives in the db as a `TIMESTAMP`, but we want to summarize some value such as transactions by day → Casting the `TIMESTAMP` to a `DATE`

In [ ]:
SELECT tx_timestamp::date, count(transactions) as num_transactions
FROM ...
GROUP BY 1
;

Assemble year, month, day into a date → use the concatenation operator `||` (double pipe) or concat function and then cast the result to a `DATE` .

Another way to convert between string and dates is by using the date function

In [ ]:
(year || '-' || month|| '-' || day)::date

cast(concat(year, '-', month, '-', day) as date)

date(concat(year, '-', month, '-', day))